# 1 - Parameters and `struphy.main`

Struphy is a collection of "models". Each model is a parallelized simulation code for a physics model described py a set partial differntial equations (PDEs).
Check the documentation for a [list of currently available models](https://struphy.pages.mpcdf.de/struphy/sections/models.html).
A model is launched through its parameter file, where all simulation parameters can be specified.

Model parameter files are Python scripts (.py) that can be executed with the Python interpreter.
For each `MODEL`, the default parameter file can be generated from the console:

```
struphy params MODEL
```

This will create a file `params_MODEL.py` in the current working directory. To run the model type

```
python3 params_MODEL.py
```

One can modify the parameter file to launch a specific simulation.
For example, the parameter file of the model [Vlasov](https://struphy.pages.mpcdf.de/struphy/sections/subsections/models_toy.html#struphy.models.toy.Vlasov) can be generated from

```
struphy params Vlasov
```

To see its contents, open the file in your preferred editor or type

```
cat params_Vlasov.py
```

We shall discuss this parameter file in what follows. Parameter files of all models have a similar structure.

## Part 1: Imports

In [ ]:
from struphy import main
from struphy.fields_background import equils
from struphy.geometry import domains
from struphy.initial import perturbations
from struphy.io.options import BaseUnits, DerhamOptions, EnvironmentOptions, FieldsBackground, Time
from struphy.kinetic_background import maxwellians

# import model, set verbosity
from struphy.models.toy import Vlasov
from struphy.pic.utilities import (
    BinningPlot,
    BoundaryParameters,
    KernelDensityPlot,
    LoadingParameters,
    WeightsParameters,
)
from struphy.topology import grids

All parameter files import the modules listed above, even though some of them might not be needed in a specific model. 
The last import imports the model itself.

## Part 2: Generic options

The following lines refer to options that can be set for any model. These are:

* Environment options (paths, saving, domain cloning)
* Model units
* Time options
* Problem geometry (mapped domain)
* Static background (equilibrium) 
* Grid 
* Derham complex

Check the respective classes for possible options.

In [ ]:
# environment options
env = EnvironmentOptions()

# units
base_units = BaseUnits()

# time stepping
time_opts = Time(dt=0.2, Tend=10.0)

# geometry
l1 = -5.0
r1 = 5.0
l2 = -7.0
r2 = 7.0
l3 = -1.0
r3 = 1.0
domain = domains.Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

# fluid equilibrium (can be used as part of initial conditions)
equil = None

# grid
grid = grids.TensorProductGrid()

# derham options
derham_opts = DerhamOptions()

## Part 3: Model instance

A model has a predefined number of "species". Each species is a collection of "variables", which are the unknowns of the model.
In the parameter file, a light-weight instance of the model is created, without allocating memory. 
The light-weight instance is used to set some model-specific parameters for each species.

For instance, for each species one can set its charge- and mass number:

In [ ]:
# light-weight model instance
model = Vlasov()

# species parameters
model.kinetic_ions.set_phys_params(charge_number=3, mass_number=12)

In case of a kinetic species, one can also set parameters regarding marker drawing, box sorting and data saving: 

In [ ]:
loading_params = LoadingParameters(Np=15)
weights_params = WeightsParameters()
boundary_params = BoundaryParameters(bc=("reflect", "reflect", "periodic"))
model.kinetic_ions.set_markers(
    loading_params=loading_params, weights_params=weights_params, boundary_params=boundary_params
)

model.kinetic_ions.set_sorting_boxes()
model.kinetic_ions.set_save_data(n_markers=1.0)

## Part 4: Propagator options

A model is a collection of "propagators" which perform the time stepping. Each propagator refers to part of a single time step and can be viewed as a splitting step. For instance, if only a single propagator is present in a model, then all variables of the model are updated by this propagator. If two or more propagators are present, they are executed in sequence, according to the chosen splitting algorithm (Lie-Trotter, Strang, etc.).

Each propagator has options that can be set in the parameter file as follows:

In [ ]:
# propagator options
model.propagators.push_vxb.options = model.propagators.push_vxb.Options()
model.propagators.push_eta.options = model.propagators.push_eta.Options()

## Part 5: Initial conditions

One can use the methods `Variable.add_background()` and `Variable.add_perturbation()` to set initial conditions for each variable of a species. Variables that are not specified are intialized as zero.

In [ ]:
# initial conditions (background + perturbation)
perturbation = None

background = maxwellians.Maxwellian3D(n=(1.0, perturbation))
model.kinetic_ions.var.add_background(background)

## Part 6: `main.run`

In the final part of the parameter file, the `main.run` command is invoked. This command will allocate memory and run the specified simulation. The run command is not executed when the parameter file is imported in another Python script. The `verbose` flag controls the screen output during the simulation run.

In [ ]:
verbose = True

main.run(
    model,
    params_path=None,
    env=env,
    base_units=base_units,
    time_opts=time_opts,
    domain=domain,
    equil=equil,
    grid=grid,
    derham_opts=derham_opts,
    verbose=verbose,
)

## Post processing: `main.pproc`

Aside from `run`, the Struphy `main` module has also a `pproc` routine for post-processing of raw simulation data:

In [ ]:
import os

path = os.path.join(os.getcwd(), "sim_1")

main.pproc(path)

One can also post-process directly from the console:

```
struphy pproc sim_1
```

Type 

```
struphy pproc -h
```

for more info on this command.

## Loading data: `main.load_data`

After post-processing, the generated data can be loaded via `main.load_data`. This function returns a `SimData` object, which you can inspect to get further info on possible data to load.

In [ ]:
simdata = main.load_data(path)

## Plotting particle orbits

In this example, for the species `kinetic_ions` some particle orbits have been saved. Under `simdata.orbits[<species_name>]` one finds a three-dimensional numpy array; the first index refers to the time step, the second index to the particle and the third index to the particel attribute. The first three attributes are the particle positions, followed by the velocities and the (initial and time-dependent) weights.

In [ ]:
orbits = simdata.orbits["kinetic_ions"]

Nt = simdata.Nt["kinetic_ions"]
Np = simdata.Np["kinetic_ions"]
Nattr = simdata.Nattr["kinetic_ions"]

Let us plot the orbits:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

fig = plt.figure()
ax = fig.gca()

colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]

# create alpha for color scaling
Tend = time_opts.Tend
alpha = np.linspace(1.0, 0.0, Nt + 1)

# loop through particles, plot all time steps
for i in range(Np):
    ax.scatter(orbits[:, i, 0], orbits[:, i, 1], c=colors[i % 4], alpha=alpha)

ax.plot([l1, l1], [l2, r2], "k")
ax.plot([r1, r1], [l2, r2], "k")
ax.plot([l1, r1], [l2, l2], "k")
ax.plot([l1, r1], [r2, r2], "k")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim(-6.5, 6.5)
ax.set_ylim(-9, 9)
ax.set_title(f"{int(Nt - 1)} time steps (full color at t=0)");